In [1]:
from text2sql.dataset.utils import process_wikisql_modified_dataset

dataset = process_wikisql_modified_dataset(
    download_folder="./data",
    split="validation",
    force=False
)

/home/anindya/miniconda3/envs/deep/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 8421/8421 [00:33<00:00, 251.80it/s]
2024-08-06 13:48:15,685 - [DATA-UTILS] - INFO - Finished processing the WikiSQL validation dataset.


In [3]:
dataset[0]["sql"]

'SELECT Position FROM table_1_10015132_11 WHERE School_Club_Team = "Butler" CC (KS)'

In [5]:
import sqlite3

conn = sqlite3.connect("data/wikisql/validation/validation.db")
cursor = conn.cursor()

In [6]:
sql = 'SELECT Position FROM table_1_10015132_11 WHERE School_Club_Team = "Butler CC (KS)"'
cursor.execute(sql)
cursor.fetchall()


[('Guard-Forward',),
 ('Guard-Forward',),
 ('Guard-Forward',),
 ('Guard-Forward',),
 ('Guard-Forward',),
 ('Guard-Forward',)]

In [8]:
cursor.execute("SELECT * FROM table_1_10015132_11")
cursor.fetchall()

# make a simple table to visualize results
import pandas as pd

cursor.execute("SELECT * FROM table_1_10015132_11")
rows = cursor.fetchall()
columns = [description[0] for description in cursor.description]

df = pd.DataFrame(rows, columns=columns)
df


,Player,No_,Nationality,Position,Years_in_Toronto,School_Club_Team
0,Antonio Lang,21,United States,Guard-Forward,1999-2000,Duke
1,Voshon Lenard,2,United States,Guard,2002-03,Minnesota
2,Martin Lewis,"32, 44",United States,Guard-Forward,1996-97,Butler CC (KS)
3,Brad Lohaus,33,United States,Forward-Center,1996,Iowa
4,Art Long,42,United States,Forward-Center,2002-03,Cincinnati
5,John Long,25,United States,Guard,1996-97,Detroit
6,Kyle Lowry,3,United States,Guard,2012-Present,Villanova
7,Antonio Lang,21,United States,Guard-Forward,1999-2000,Duke
8,Voshon Lenard,2,United States,Guard,2002-03,Minnesota
9,Martin Lewis,"32, 44",United States,Guard-Forward,1996-97,Butler CC (KS)


In [9]:
import json 


len(json.load(open("data/bird/train/train.json", "r")))

9428

In [10]:
from datasets import load_dataset

d = load_dataset("Salesforce/wikisql")["train"]

In [13]:
d[0]["sql"]["human_readable"]

'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'

In [14]:
import re

def quote_sql_values(sql_query):
    # This regex matches values after '=' and before spaces or closing parentheses, assuming they are not already quoted
    def replacer(match):
        value = match.group(2)
        # Check if the value is already quoted
        if not (value.startswith("'") and value.endswith("'")) and not (value.startswith('"') and value.endswith('"')):
            value = f"'{value}'"
        return f"{match.group(1)}{value}"

    # Find and quote values in WHERE clauses
    sql_query = re.sub(r'(\s=\s)([^\'"\s)(]+)', replacer, sql_query)

    # Ensure column names with spaces or special characters are enclosed in square brackets
    sql_query = re.sub(r'\b(\w+\s\w+)\b', r'[\1]', sql_query)

    return sql_query

In [15]:
sql_query = 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'
corrected_sql_query = quote_sql_values(sql_query)
print(corrected_sql_query)

[SELECT Notes] [FROM table] [WHERE Current] slogan = 'SOUTH' AUSTRALIA


In [6]:
dataset[0]["table"]

'table_1_10015132_11'

In [5]:
cursor.execute("PRAGMA table_info(table_1_10015132_11)")
columns = cursor.fetchall()
columns

[(0, 'Player', 'TEXT', 0, None, 0),
 (1, 'No_', 'TEXT', 0, None, 0),
 (2, 'Nationality', 'TEXT', 0, None, 0),
 (3, 'Position', 'TEXT', 0, None, 0),
 (4, 'Years_in_Toronto', 'TEXT', 0, None, 0),
 (5, 'School_Club_Team', 'TEXT', 0, None, 0)]

In [16]:
len(d)

56355

In [17]:
len(dataset)

8421

In [2]:
len(dataset)

8421

In [4]:
dataset[0]

{'question': 'What position does the player who played for butler cc (ks) play?',
 'table': 'table_1_10015132_11',
 'db_path': 'data/wikisql/validation/validation.db',
 'sql': 'SELECT Position FROM table_1_10015132_11 WHERE School/Club Team = Butler CC (KS)'}

In [5]:
# TODO: the name of the columns in the table and in the SQL are not in sync 
# TODO: I also need to test once I handle the above issue to see
# if things are retrieved correctly or not


import sqlite3

conn = sqlite3.connect("data/wikisql/validation/validation.db")
cursor = conn.cursor()


In [9]:
# list out all the columns of table: table_1_10015132_11

cursor.execute("PRAGMA table_info(table_1_10015132_11)")
columns = cursor.fetchall()
columns

[(0, 'Player', 'TEXT', 0, None, 0),
 (1, 'No_', 'TEXT', 0, None, 0),
 (2, 'Nationality', 'TEXT', 0, None, 0),
 (3, 'Position', 'TEXT', 0, None, 0),
 (4, 'Years_in_Toronto', 'TEXT', 0, None, 0),
 (5, 'School_Club_Team', 'TEXT', 0, None, 0)]

In [15]:
import sqlparse
from difflib import get_close_matches

sql_query = dataset[0]["sql"]
actual_columns = [col[1] for col in columns]
parsed = sqlparse.parse(sql_query)[0]


In [17]:
identifiers = [token for token in parsed.tokens if isinstance(token, sqlparse.sql.Identifier)]
identifiers

[<Identifier 'table_...' at 0x7FBABC9887C0>]

In [18]:
corrected_query = sql_query
for identifier in identifiers:
    col_name = str(identifier).strip()
    
    close_matches = get_close_matches(col_name, actual_columns, n=1, cutoff=0.6)
    if close_matches:
        correct_name = close_matches[0]
        corrected_query = corrected_query.replace(col_name, correct_name)

In [19]:
corrected_query

'SELECT Position FROM table_1_10015132_11 WHERE School/Club Team = Butler CC (KS)'

In [21]:
import sqlparse
from difflib import get_close_matches

def correct_column_names(column_metadata, sql_query):
    # Extract actual column names from metadata
    actual_columns = [col[1] for col in column_metadata]
    
    # Parse the SQL query
    parsed = sqlparse.parse(sql_query)[0]
    
    # Extract the identifiers from the SQL query
    identifiers = [token for token in parsed.tokens if isinstance(token, sqlparse.sql.Identifier)]
    
    # Correct the column names
    corrected_query = sql_query
    for identifier in identifiers:
        col_name = str(identifier).strip()
        close_matches = get_close_matches(col_name, actual_columns, n=1, cutoff=0.6)
        if close_matches:
            correct_name = close_matches[0]
            corrected_query = corrected_query.replace(col_name, correct_name)
    
    return corrected_query

In [22]:
# Example usage
column_metadata = [
    (0, 'Player', 'TEXT', 0, None, 0),
    (1, 'No_', 'TEXT', 0, None, 0),
    (2, 'Nationality', 'TEXT', 0, None, 0),
    (3, 'Position', 'TEXT', 0, None, 0),
    (4, 'Years_in_Toronto', 'TEXT', 0, None, 0),
    (5, 'School_Club_Team', 'TEXT', 0, None, 0)
]

sql_query = "SELECT Position FROM table_1_10015132_11 WHERE School/Club Team = 'Butler CC (KS)'"
column_metadata = columns 

corrected_sql = correct_column_names(column_metadata, sql_query)
print(corrected_sql)

SELECT Position FROM table_1_10015132_11 WHERE School/Club Team = 'Butler CC (KS)'


In [24]:
sql = "SELECT Position FROM table_1_10015132_11 WHERE School_Club_Team = 'Butler CC (KS)'"
cursor.execute(sql)

cursor.fetchall()


[('Guard-Forward',),
 ('Guard-Forward',),
 ('Guard-Forward',),
 ('Guard-Forward',),
 ('Guard-Forward',)]

In [25]:
sql = "SELECT * FROM table_1_10015132_11"
cursor.execute(sql)

cursor.fetchall()

[('Antonio Lang', '21', 'United States', 'Guard-Forward', '1999-2000', 'Duke'),
 ('Voshon Lenard', '2', 'United States', 'Guard', '2002-03', 'Minnesota'),
 ('Martin Lewis',
  '32, 44',
  'United States',
  'Guard-Forward',
  '1996-97',
  'Butler CC (KS)'),
 ('Brad Lohaus', '33', 'United States', 'Forward-Center', '1996', 'Iowa'),
 ('Art Long',
  '42',
  'United States',
  'Forward-Center',
  '2002-03',
  'Cincinnati'),
 ('John Long', '25', 'United States', 'Guard', '1996-97', 'Detroit'),
 ('Kyle Lowry', '3', 'United States', 'Guard', '2012-Present', 'Villanova'),
 ('Antonio Lang', '21', 'United States', 'Guard-Forward', '1999-2000', 'Duke'),
 ('Voshon Lenard', '2', 'United States', 'Guard', '2002-03', 'Minnesota'),
 ('Martin Lewis',
  '32, 44',
  'United States',
  'Guard-Forward',
  '1996-97',
  'Butler CC (KS)'),
 ('Brad Lohaus', '33', 'United States', 'Forward-Center', '1996', 'Iowa'),
 ('Art Long',
  '42',
  'United States',
  'Forward-Center',
  '2002-03',
  'Cincinnati'),
 ('John

In [ ]:
import os 
from zipfile import ZipFile

with ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall(download_path)

os.remove(zip_file)

In [5]:
from text2sql.dataset.utils import process_wikisql_modified_dataset

train_dataset = process_wikisql_modified_dataset(
    download_folder="./data",
    split="train",
    force=False
)

  9%|▉         | 5341/56355 [00:21<03:41, 230.66it/s]2024-08-06 10:18:54,008 - [DATA-UTILS] - ERROR - Failed to create table table_1_169766_13. Error: near "Returning": syntax error
2024-08-06 10:18:54,009 - [DATA-UTILS] - ERROR - Failed to create table table_1_169766_13. Error: near "Returning": syntax error
2024-08-06 10:18:54,009 - [DATA-UTILS] - ERROR - Failed to create table table_1_169766_13. Error: near "Returning": syntax error
2024-08-06 10:18:54,010 - [DATA-UTILS] - ERROR - Failed to create table table_1_169766_13. Error: near "Returning": syntax error
2024-08-06 10:18:54,011 - [DATA-UTILS] - ERROR - Failed to create table table_1_169766_13. Error: near "Returning": syntax error
 54%|█████▍    | 30696/56355 [02:50<03:39, 116.87it/s]2024-08-06 10:21:23,247 - [DATA-UTILS] - ERROR - Failed to create table table_2_169743_3. Error: near "Returning": syntax error
2024-08-06 10:21:23,250 - [DATA-UTILS] - ERROR - Failed to create table table_2_169743_3. Error: near "Returning": synta

In [1]:
from text2sql.dataset.utils import download_and_process_bird_dataset

download_and_process_bird_dataset(
    split="train",
    download_folder="./data",
    force=False
)

/home/anindya/miniconda3/envs/deep/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-06 10:17:40,873 - [DATA-UTILS] - INFO - => Dataset for training [BirdBench trainset] already exists.


In [2]:
from text2sql.dataset.utils import download_and_process_bird_dataset

download_and_process_bird_dataset(
    split="validation",
    download_folder="./data",
    force=False
)

2024-08-06 10:17:40,880 - [DATA-UTILS] - INFO - => Dataset for evaluation [BirdBench devset] already exists.


In [16]:
from pathlib import Path

download_path = Path("./data") / "bird" / "train"
zip_file = download_path / "train.zip"
inner_zips = download_path / "train"

inner_zip_file = download_path / "train_databases.zip"
unzip_dir = download_path / "train_databases"
macosx_dir = download_path / "__MACOSX"
inner_macosx_dir = download_path / "train_databases" / "__MACOSX"

In [ ]:
inner_train_databases = unzip_dir / "train_databases"

In [8]:
import os 
from zipfile import ZipFile

with ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall(download_path)

os.remove(zip_file)

In [10]:
import shutil

if macosx_dir.exists():
    shutil.rmtree(macosx_dir, ignore_errors=True)

In [12]:
# Now move all contents from download_path / train to download_path

for file in inner_zips.iterdir():
    shutil.move(file, download_path)

In [13]:
# Now remove download_path / train
shutil.rmtree(inner_zips, ignore_errors=True)

In [15]:
# Now extract download_path / train_databases.zip to download_path / train_databases
with ZipFile(inner_zip_file, "r") as zip_ref:
    zip_ref.extractall(unzip_dir)

os.remove(inner_zip_file)

In [17]:
# Now delete __MACOSX inside download_path / train_databases

if inner_macosx_dir.exists():
    shutil.rmtree(inner_macosx_dir, ignore_errors=True)

In [19]:
inner_train_databases = unzip_dir / "train_databases"

# first rename download_path / train_databases to download_path / tmp
# then move the folder inside download_path / tmp to download_path 
# and delete download_path / tmp

tmp_dir = download_path / "tmp"
inner_train_databases.rename(tmp_dir)

PosixPath('data/bird/train/tmp')

In [22]:
if unzip_dir.exists():
    shutil.rmtree(unzip_dir, ignore_errors=True)

In [23]:
# Now rename tmp back to train_databases
tmp_dir.rename(unzip_dir)

PosixPath('data/bird/train/train_databases')